In [66]:
from pyspark.sql.types import *
from pyspark.sql import *
from pyspark import SQLContext, HiveContext, SparkContext
import os
import sys
import argparse
import logging
import datetime

## Definições para o contexto do Spark

In [54]:
#sc.stop()

In [51]:
sc = SparkContext()

In [52]:
sqlContext = SQLContext(sc)

In [53]:
sqlContext.setConf('spark.sql.parquet.compression.codec','snappy')

In [57]:
hive_context = HiveContext(sc)

In [55]:
hive_database = "vc"

In [23]:
#rdd = sc.parallelize(range(1000))

In [24]:
#rdd.takeSample(False,5)

In [5]:
#logging tem 5 niveis de erro
#debug info warning error critical

## Definições de mensagem de erro

In [62]:
logger = logging.getLogger("")

In [29]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s: BDF_INGESTION %(message)s')

In [30]:
def setError(msg):
    logger.error(msg)
    sys.exit(1)

In [63]:
def setInfo(msg):
    logger.info(msg)

In [10]:
logging.debug('Este é um erro de debug')
logging.debug('Este é um erro de debug 11111')

# Funções principais

In [64]:
smart_table_name = 'movto_valido'
setInfo("## Parametro recebido: {}".format(smart_table_name))

In [147]:
# Valida data de carga das bases
# O retorno da função será:
#  Lista 0 - Status 
#    OK  - Data da Carga igual a esperada (dia -1d)
#    NOK - Data da Carga diferente da esperada (dia -1d)
#  Lista 1 - nome da tabela
#  Lista 2 - Data da Carga encontrada na base
#    Preencher somente em caso de 'NOK'
def valida_dta_referencia():
    fl_data_ref = [[],[],[]]
    setInfo("## Validação de data de referencia")
    dt_ref_t254 = 0
    fl_data_ref[0].append('NOK')
    return fl_data_ref
    

In [65]:
# Movimentos Validos
# Essa base é gerada através da tabela vcdt254.
# A tabela se encontra na database "vc"
def movto_valido():
    setInfo("## Fase 00 - Valida data das cargas das bases")
    setInfo("## Fase 01 - Seleciona publico < 700.00 ##")
    schema_mvto_validos = StructType([
        StructField("nr_prop",         StringType(),True),
        StructField("nr_parc_prop",    StringType(),True),
        StructField("nr_apol",         StringType(),True),
        StructField("nr_cerf_segr",    StringType(),True),
        StructField("cd_segd",         StringType(),True),
        StructField("nr_sequ_movi",    StringType(),True),
        StructField("cd_prod_segd",    StringType(),True),
        StructField("cd_subp_segd",    StringType(),True),
        StructField("dt_oper_cntb",    StringType(),True),
        StructField("dt_gera_cntb",    StringType(),True),
        StructField("cd_even_cntb",    StringType(),True),
        StructField("cd_idef_oper",    StringType(),True),
        StructField("nr_cnta_cntb",    StringType(),True),
        StructField("tp_cntb",         StringType(),True),
        StructField("vl_lanc_cntb",    DecimalType(17,2),True),
        StructField("tx_info_extt_ha", StringType(),True),
        StructField("dh_ulti_atlz",    TimestampType(),True),
        StructField("dat_ref_carga",   StringType(),True)
    ])
    
    df_fase01 = hive_context.createDataFrame(sc.emptyRDD(),schema=schema_movto_validos)
    
    df_fase01 = SQLContext.sql("""select  
                                     nr_prop
                                    ,nr_parc_prop
                                    ,nr_apol
                                    ,nr_cerf_segr
                                    ,cd_segd
                                    ,nr_sequ_movi  
                                    ,cd_prod_segd  
                                    ,cd_subp_segd  
                                    ,dt_oper_cntb  
                                    ,dt_gera_cntb  
                                    ,cd_even_cntb  
                                    ,cd_idef_oper  
                                    ,nr_cnta_cntb  
                                    ,tp_cntb
                                    ,vl_lanc_cntb
                                    ,tx_info_extt_ha
                                    ,dh_ulti_atlz
                                    ,dat_ref_carga
                                  from vcdt254""".format(db=hive_database))
    df_fase01.cache()
    
    print("## Fase 02 - Identifica contas 300.000 com saldo igual a zero ##")
    print("## Fase 03 - Retira do publico da Fase 01 o publico encontrado na Fase 02 ##")
    print("## Fase 04 - Formata smart table analitica de movimentos validos com data de geracao")

In [14]:
def comissoes():
    print("##Entrou em comissoes")

In [15]:
def opcao_03():
    print("##Entrou na opcao 03")

## Aqui sera a funcao principal

In [168]:
dt_tmp = datetime.datetime.now()
dt_base_carga = dt_tmp.strftime("%Y") + "-" + dt_tmp.strftime("%m") + "-" + dt_tmp.strftime("%d")
dia = dt_tmp.strftime("%d")
print(dia)
print(dt_base_carga)

15
2019-05-15


In [164]:
#Formata data no padrao do hive
dt_tmp = datetime.datetime.now()
dt_base_carga = dt_tmp.strftime("%Y") + "-" + dt_tmp.strftime("%m") + "-" + dt_tmp.strftime("%d")
print(dt_base_carga)
fl_data_ref = valida_dta_referencia()

if 'NOK' in fl_data_ref[0]:
    setInfo("## Bases Origem com datas de carga diferente!!! ##")
    setInfo("## Data base de carga: {} ##".format(dt_base_carga))
    setInfo("")
    sys.exit(1)

if smart_table_name == "movto_valido":
    movto_valido()
elif smart_table_name == "comissoes":
    comissoes()
elif smart_table_name == "opcao_03":
    opcao_03()
else:
    print("## Smart Table Invalida ou nao informada. ##")

2019-05-15


SystemExit: 1

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3299: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
